In [7]:
import os 
import pandas as pd
import shutil
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

seed = 5
# directories
dirpath = os.getcwd()
img_dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_processing\validate_watershed\watershed_data\masked_frame')
arr_dir =  os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_processing\validate_watershed\watershed_data\array')
data2_dir = os.path.join(dirpath, r"ws_classify\array\data")
data1_dir = os.path.join(dirpath, r"ws_classify\masked_frame\data")
# class sest df
class_df = pd.read_csv(os.path.join(os.path.normpath(dirpath + os.sep + os.pardir), r'udder_processing\validate_watershed\survey_results.csv'))
class_dict = {0:"bad", 1:"good"}

def mk_dest_dir(data_dir , file_set, file_class):
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    set_dir = os.path.join(data_dir, file_set)
    if not os.path.exists(set_dir):
        os.mkdir(set_dir)
    class_dir = os.path.join(set_dir, file_class)
    if not os.path.exists(class_dir):
        os.mkdir(class_dir)

In [8]:
splits = 5
fold_df = pd.DataFrame(columns=["cow", "fold_group"])
x = np.array(list(class_df.filename))
group_cows = np.array(list(class_df.cow))
group_class = np.array(list(class_df.img_class))
sgkf = StratifiedGroupKFold(n_splits=5)
sgkf.get_n_splits(x, group_class)
test_list = []
fold_list = []
for i, (train_index, test_index) in enumerate(sgkf.split(x, group_class, group_cows)):
    test_cows = np.unique(group_cows[test_index])
    test_list.extend(list(test_cows))
    fold_list.extend([i]*len(test_cows))
temp_df = pd.DataFrame({"cow": test_list, "fold_group": fold_list})
fold_df = pd.concat([fold_df , temp_df], axis = 0, ignore_index = True)
    
merged = pd.merge(class_df, fold_df, on = "cow")

split_groups = list(range(splits))
np.random.seed(seed)
np.random.shuffle(split_groups)
proportions = np.multiply(splits,[0.6, 0.2, 0.2]).astype(int)
set_list = ["train"] * proportions[0] + ["val"] * proportions[1] + ["test"] * proportions[2]
set_df = pd.DataFrame({"fold_group":split_groups, "set_name": set_list})

merged2 = pd.merge(merged, set_df, on = "fold_group")
merged2.to_csv("wsclass_sets.csv", index = False)

In [9]:
merged_group = merged[["fold_group", "cow", "img_class"]].groupby(["fold_group", "cow"]).agg(["sum", "count"]).reset_index()
merged_group.columns = ["_".join(name) if len(name[1]) >1 else name[0] for name in merged_group.columns]
# merged_group.sort_values(by =["fold_group", "cow"]).reset_index(drop = True)

In [10]:
# verify cows do not reapeatd
train_cows = set(merged2[merged2.set_name == "train"]["cow"].values)
test_cows = set(merged2[merged2.set_name == "test"]["cow"].values)
val_cows = set(merged2[merged2.set_name == "val"]["cow"].values)
print(val_cows.intersection(test_cows, train_cows))
print(train_cows.intersection(test_cows, val_cows))
print(test_cows.intersection(val_cows, train_cows))

set()
set()
set()


In [11]:
# class sest df
sets_df = pd.read_csv("wsclass_sets.csv")

In [13]:
for file in sets_df.filename:
    file_line = sets_df[sets_df.filename == file]
    src1 = os.path.join(img_dir, file +".png")
    src2 = os.path.join(arr_dir, file +".npy")
    # find/create dest directoy
    file_set = file_line["set_name"].values[0]
    file_class = class_dict[file_line["img_class"].values[0]]
    mk_dest_dir(data1_dir , file_set, file_class)
    mk_dest_dir(data2_dir , file_set, file_class)
    dest1_dir = os.path.join(data1_dir , file_set, file_class)
    dest2_dir = os.path.join(data2_dir , file_set, file_class)
    # print(dest1_dir)
    shutil.copy(src1, dest1_dir)
    shutil.copy(src2, dest2_dir)